In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

In [2]:
my_run = BuildStockQuery(db_name='euss-final',
                        table_name='euss_res_final_2018_550k_20220901',
                        workgroup='largeee',
                        buildstock_type='resstock')

INFO:buildstock_query.query_core:Loading euss_res_final_2018_550k_20220901 ...
INFO:buildstock_query.query_core:12 queries cache read from .bsq_cache/euss_res_final_2018_550k_20220901_query_cache.pkl.
INFO:buildstock_query.query_core:12 queries cache is updated.
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


         Fail  Unapplicaple  Success     Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                      
0        1084             0   548916  550000        0.0        0         0   
1           1         17092   531823  548916       96.9     6097       949   
2           2         12117   536797  548916       97.8     6527       879   
3           0           656   548260  548916       99.9      807     29954   
4           0           656   548260  548916       99.9        4     20705   
5         107         16653   532156  548916       97.0      819     47818   
6           1          3584   545331  548916       99.3       31      2757   
7           2        148335   400579  548916       73.0     4169      7603   
8           3             0   548913  548916      100.0        2      6456   
9           0             0   548916  548916      100.0        1      5443   
10          0             0   548916  548916      100.0        1

INFO:buildstock_query.query_core:12 queries cache saved to .bsq_cache/euss_res_final_2018_550k_20220901_query_cache.pkl


In [5]:
baseline_agg = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'])
baseline_agg


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu
0,Mobile Home,703,9.361996e+06,4.506122e+08
1,Multi-Family with 2 - 4 Units,446,5.939474e+06,1.385683e+08
2,Multi-Family with 5+ Units,1189,1.583416e+07,4.099084e+08
3,Single-Family Attached,530,7.058119e+06,2.094629e+08
4,Single-Family Detached,7121,9.483182e+07,4.413546e+09


In [ ]:
my_run.save_cache()

In [7]:
all_enduses = my_run.get_cols(my_run.ts_table)
all_enduses = [str(x) for x in all_enduses if 'end_use__' in str(x)]
all_enduses

['euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__ceiling_fan__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__clothes_dryer__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__clothes_washer__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__cooling__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__cooling_fans_pumps__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__dishwasher__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__freezer__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__heating__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__heating_fans_pumps__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__heating_heat_pump_backup__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__hot_tub_heater__kwh',
 'euss_res

In [15]:
ts_agg = my_run.agg.aggregate_timeseries(enduses=['fuel_use__electricity__total__kwh'],
                                         # restrict=[('build_existing_model.state', ['CO'])],
                                         timestamp_grouping_func='month',
                                         group_by=['geometry_building_type_recs', 'build_existing_model.state'],
                                         get_query_only=True)
print(ts_agg)

INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


SELECT euss_res_final_2018_550k_20220901_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, euss_res_final_2018_550k_20220901_baseline."build_existing_model.state" AS state, date_trunc('month', date_add('second', -900, euss_res_final_2018_550k_20220901_timeseries.time)) AS time, count(distinct(euss_res_final_2018_550k_20220901_timeseries.building_id)) AS sample_count, (count(distinct(euss_res_final_2018_550k_20220901_timeseries.building_id)) * sum(euss_res_final_2018_550k_20220901_baseline."build_existing_model.sample_weight")) / sum(1) AS units_count, sum(1) / count(distinct(euss_res_final_2018_550k_20220901_timeseries.building_id)) AS rows_per_sample, sum(euss_res_final_2018_550k_20220901_timeseries.fuel_use__electricity__total__kwh * euss_res_final_2018_550k_20220901_baseline."build_existing_model.sample_weight") AS fuel_use__electricity__total__kwh 
FROM euss_res_final_2018_550k_20220901_timeseries JOIN euss_res_final_2018_550k_20220901_base

In [14]:
ts_agg

,geometry_building_type_recs,state,time,sample_count,units_count,rows_per_sample,fuel_use__electricity__total__kwh
0,Mobile Home,AL,2018-01-01,1233,298547.538693,2976,5.749913e+08
1,Mobile Home,AL,2018-02-01,1233,298547.538693,2688,2.702862e+08
2,Mobile Home,AL,2018-03-01,1233,298547.538693,2976,2.812449e+08
3,Mobile Home,AL,2018-04-01,1233,298547.538693,2880,2.391579e+08
4,Mobile Home,AL,2018-05-01,1233,298547.538693,2976,2.993396e+08
...,...,...,...,...,...,...,...
2923,Single-Family Detached,WY,2018-08-01,725,175544.984227,2976,1.149405e+08
2924,Single-Family Detached,WY,2018-09-01,725,175544.984227,2880,1.090155e+08
2925,Single-Family Detached,WY,2018-10-01,725,175544.984227,2976,1.492910e+08
2926,Single-Family Detached,WY,2018-11-01,725,175544.984227,2880,2.082698e+08


In [7]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings
0,2018-01-01 00:15:00,9989,1.330256e+08,7.148833e+07,1.966400e+07
1,2018-01-01 00:30:00,9989,1.330256e+08,7.227770e+07,1.960490e+07
2,2018-01-01 00:45:00,9989,1.330256e+08,7.189331e+07,1.947231e+07
3,2018-01-01 01:00:00,9989,1.330256e+08,7.202095e+07,1.937339e+07
4,2018-01-01 01:15:00,9989,1.330256e+08,7.083665e+07,1.915390e+07
...,...,...,...,...,...
35035,2018-12-31 23:00:00,9989,1.330256e+08,4.153427e+07,5.265636e+06
35036,2018-12-31 23:15:00,9989,1.330256e+08,3.732818e+07,5.140162e+06
35037,2018-12-31 23:30:00,9989,1.330256e+08,3.620637e+07,5.337922e+06
35038,2018-12-31 23:45:00,9989,1.330256e+08,3.537120e+07,5.329013e+06


In [8]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             group_by=['geometry_building_type_recs'],
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings,geometry_building_type_recs
0,2018-01-01 00:15:00,703,9.361996e+06,1.098386e+07,2.682538e+06,Mobile Home
1,2018-01-01 00:30:00,703,9.361996e+06,1.093122e+07,2.634476e+06,Mobile Home
2,2018-01-01 00:45:00,703,9.361996e+06,1.077442e+07,2.593965e+06,Mobile Home
3,2018-01-01 01:00:00,703,9.361996e+06,1.080007e+07,2.552429e+06,Mobile Home
4,2018-01-01 01:15:00,703,9.361996e+06,1.059866e+07,2.516619e+06,Mobile Home
...,...,...,...,...,...,...
175195,2018-12-31 23:00:00,7121,9.483182e+07,3.171454e+07,4.143382e+06,Single-Family Detached
175196,2018-12-31 23:15:00,7121,9.483182e+07,2.811727e+07,3.993204e+06,Single-Family Detached
175197,2018-12-31 23:30:00,7121,9.483182e+07,2.729993e+07,4.172387e+06,Single-Family Detached
175198,2018-12-31 23:45:00,7121,9.483182e+07,2.672484e+07,4.160202e+06,Single-Family Detached


In [9]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             group_by=['geometry_building_type_recs'],
                             get_quartiles=True,
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings,fuel_use__electricity__total__kwh__savings__quartiles,geometry_building_type_recs
0,2018-01-01 00:15:00,703,9.361996e+06,1.098386e+07,2.682538e+06,"[-0.05399999999999999, 0.0, 0.0155899999999999...",Mobile Home
1,2018-01-01 00:30:00,703,9.361996e+06,1.093122e+07,2.634476e+06,"[-0.031, 0.0, 0.0158356401384083, 0.1711655303...",Mobile Home
2,2018-01-01 00:45:00,703,9.361996e+06,1.077442e+07,2.593965e+06,"[-0.03300000000000003, 0.0, 0.0154038281979458...",Mobile Home
3,2018-01-01 01:00:00,703,9.361996e+06,1.080007e+07,2.552429e+06,"[-0.027999999999999997, 0.0, 0.016376282051282...",Mobile Home
4,2018-01-01 01:15:00,703,9.361996e+06,1.059866e+07,2.516619e+06,"[-0.024999999999999994, 0.0, 0.015727272727272...",Mobile Home
...,...,...,...,...,...,...,...
175195,2018-12-31 23:00:00,7121,9.483182e+07,3.171454e+07,4.143382e+06,"[-0.473, -0.020270068826590562, 0.0, 0.0040731...",Single-Family Detached
175196,2018-12-31 23:15:00,7121,9.483182e+07,2.811727e+07,3.993204e+06,"[-0.489, -0.02042967741935484, 0.0, 0.00376620...",Single-Family Detached
175197,2018-12-31 23:30:00,7121,9.483182e+07,2.729993e+07,4.172387e+06,"[-0.518, -0.017058074866310155, 0.0, 0.0040970...",Single-Family Detached
175198,2018-12-31 23:45:00,7121,9.483182e+07,2.672484e+07,4.160202e+06,"[-0.587, -0.017366656552689926, 0.0, 0.0041328...",Single-Family Detached
